In [11]:
from openai import OpenAI
from getpass import getpass
import json
import re

In [2]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [60]:
with open('data/check_card_info-ver-4.json', 'r', encoding='utf-8') as json_file:
    check_card_data = json.load(json_file)

with open('data/credit_card_info-ver-4.json', 'r', encoding='utf-8') as json_file:
    credit_card_data = json.load(json_file)

In [61]:
question = '''체크 카드 데이터에서 군인한테 카드 추천해봐'''

def analysis_data(check, question, model):
    '''파일과 질의를 구분한 사용자 정의 함수'''
    prompt = f'체크 카드 데이터 : {check}\nQuestion : {question}'
    response = client.chat.completions.create(model=model,
                                          messages=[{'role':'user','content':prompt}],
                                          max_tokens=500 
                                          )
    
    return response.choices[0].message.content

analysis_data(credit_card_data, question, 'gpt-3.5-turbo')

"군인을 위한 카드 추천은 다양한 요소를 고려하여 결정되어야 합니다. 하지만, 군인들에게 가장 인기 있는 카드 중 하나는 '신한카드 The CLASSIC+'일 것으로 예상됩니다. 이 카드는 1,500원당 1마일리지 적립, 1,000원당 1마일리지 적립, PP카드 무료 제공 등 다양한 혜택을 제공하며, 조직에서 근무하시는 군인들에게 유용할 수 있습니다. 또한, 국내 전 가맹점 최대 1.5% 하나머니 적립과 바우처 혜택도 포함되어 있습니다. 다만, 실제 선택 전에 자세한 내용을 확인하시고 해당 카드가 본인의 상황과 필요에 맞는지 검토해보시는 것이 좋습니다."

In [122]:
# 유의사항, 기타 항목 제거
for i in range(len(credit_card_data)):
    for k, v in credit_card_data[i].items():
        v['주요 혜택'] = {sub_k: sub_v for sub_k, sub_v in v['주요 혜택'].items() if sub_k not in ['유의사항', '기타']}


In [123]:
# 이미지 파일 명에서 코드 항목만 추출하여 코드를 검색하여 이미지를 찾을 수 있게 수정
import re

for i in range(len(credit_card_data)):
    for k, v in credit_card_data[i].items():
        file_name = v['카드 이미지'].split('\\')[1]  # 파일명 추출

        match = re.search(r'(\d+)[a-zA-Z]', file_name)
        if match:
            extracted_number = match.group(1)  # "2749"
            
            del v['카드 이미지']  # '카드 이미지' 항목 제거
            v['code'] = extracted_number  # 새로운 'code' 항목 생성

In [ ]:
# # 주요 혜택 카테고리의 대괄호(부연 설명) 제거
# for i in range(len(credit_card_data)):
#     for k, v in credit_card_data[i].items():
#         for sub_k, sub_v in v['주요 혜택'].items():
#             v['주요 혜택'][sub_k] = re.sub(r'\[.*?\]\s*', '', sub_v)
        

In [ ]:
# 주요 혜택 카테고리의 소괄호(부연 설명) 제거 (체크)
for i in range(len(check_card_data)):
    for k, v in check_card_data[i].items():
        v['연회비 상세안내'] = re.sub(r'\(.*?\)\s*', '', v['연회비 상세안내'])
        

In [ ]:
# 주요 혜택 카테고리의 소괄호(부연 설명) 제거 (신용)
for i in range(len(credit_card_data)):
    for k, v in credit_card_data[i].items():
        v['혜택 요약'] = re.sub(r'\(.*?\)\s*', '', v['혜택 요약'])

In [59]:
len(credit_card_data)

50

In [29]:
with open('data/credit_card_info-ver-4.json', 'w', encoding='utf-8') as json_file:
    json.dump(credit_card_data, json_file, ensure_ascii=False, indent=4)

In [30]:
with open('data/check_card_info-ver-4.json', 'w', encoding='utf-8') as json_file:
    json.dump(check_card_data, json_file, ensure_ascii=False, indent=4)